In [23]:
pip install gradio torch jupyter ipywidgets transformers ollama

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [25]:
import ollama
ollama.pull('mistral')

{'status': 'success'}

In [30]:
from ollama import Client
client = Client(host='http://localhost:11434')
response = client.chat(model='mistral', messages=[
  {
    'role': 'user',
    'content': 'Why is the sky blue?',
  },
])
print(response['message']['content'])

 The reason why the sky appears blue during a clear day is due to a phenomenon called Rayleigh scattering. When the sun's rays enter Earth's atmosphere, they collide with molecules and tiny particles in the air, such as nitrogen and oxygen. Blue light has a shorter wavelength and gets scattered more easily than other colors. Consequently, when we look up at the sky, we predominantly see the blue light that has been scattered, making the sky appear blue to our eyes.


In [33]:
import ollama

stream = ollama.chat(
    model='mistral',
    messages=[{'role': 'user', 'content': 'Why is the sky blue?'}],
    stream=True,
)

for chunk in stream:
  print(chunk['message']['content'], end='', flush=True)

 The color of the sky appears blue due to a phenomenon called Rayleigh scattering. As sunlight reaches Earth's atmosphere, it interacts with molecules and particles in the air, such as nitrogen and oxygen. Blue light has shorter wavelengths and gets scattered more easily than other colors because it travels in smaller, shorter waves. This scattered blue light is what we observe when looking up at the sky. However, it's essential to note that the sky doesn't always appear blue; its color can change depending on the presence of other particles or pollutants in the atmosphere. For example, sunrise and sunset skies often exhibit red, orange, or pink hues due to the scattering of longer wavelengths of light.

In [36]:
ollama.list()

{'models': [{'name': 'mistral:7b',
   'model': 'mistral:7b',
   'modified_at': '2024-04-07T15:35:48.345014632Z',
   'size': 4109865159,
   'digest': '61e88e884507ba5e06c49b40e6226884b2a16e872382c2b44a42f2d119d804a5',
   'details': {'parent_model': '',
    'format': 'gguf',
    'family': 'llama',
    'families': ['llama'],
    'parameter_size': '7B',
    'quantization_level': 'Q4_0'}}]}

In [39]:
ollama.show('mistral:7b')

{'license': '                                 Apache License\n                           Version 2.0, January 2004\n                        http://www.apache.org/licenses/\n\n   TERMS AND CONDITIONS FOR USE, REPRODUCTION, AND DISTRIBUTION\n\n   1. Definitions.\n\n      "License" shall mean the terms and conditions for use, reproduction,\n      and distribution as defined by Sections 1 through 9 of this document.\n\n      "Licensor" shall mean the copyright owner or entity authorized by\n      the copyright owner that is granting the License.\n\n      "Legal Entity" shall mean the union of the acting entity and all\n      other entities that control, are controlled by, or are under common\n      control with that entity. For the purposes of this definition,\n      "control" means (i) the power, direct or indirect, to cause the\n      direction or management of such entity, whether by contract or\n      otherwise, or (ii) ownership of fifty percent (50%) or more of the\n      outstanding

In [60]:
import gradio as gr
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, StoppingCriteria, StoppingCriteriaList, TextIteratorStreamer
from threading import Thread

def mistralChat(text):
    tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")
    model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2", torch_dtype=torch.bfloat16)
    model = model.to('cuda')
    inputs = tokenizer(text, return_tensors='pt').to(model.device)
    
    input_length = inputs.input_ids.shape[1]
    outputs = model.generate(**inputs, max_new_tokens=100, temperature=0.7, 
                            return_dict_in_generate=True,do_sample=True)
    
    tokens = outputs.sequences[0, input_length:]
    return tokenizer.decode(tokens)


In [61]:
with gr.Blocks() as server:
    with gr.Tab("LLM Inferencing"):
    
        model_input = gr.Textbox(label="Your Question:", 
                                value="What’s your question?", interactive=True)
        ask_button = gr.Button("Ask")
        model_output = gr.Textbox(label="The Answer:", interactive=False, 
                                value="Answer goes here...")
    
    ask_button.click(mistralChat, inputs=[model_input], outputs=[model_output])

server.launch(share=True)

Running on local URL:  http://127.0.0.1:7868
Running on public URL: https://c9b559173035bea6c8.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Loading checkpoint shards: 100%|██████████| 3/3 [00:09<00:00,  3.01s/it]
Traceback (most recent call last):
  File "/home/kaminaduck/code/ai-holodeck/venv/lib/python3.10/site-packages/gradio/queueing.py", line 522, in process_events
    response = await route_utils.call_process_api(
  File "/home/kaminaduck/code/ai-holodeck/venv/lib/python3.10/site-packages/gradio/route_utils.py", line 260, in call_process_api
    output = await app.get_blocks().process_api(
  File "/home/kaminaduck/code/ai-holodeck/venv/lib/python3.10/site-packages/gradio/blocks.py", line 1741, in process_api
    result = await self.call_function(
  File "/home/kaminaduck/code/ai-holodeck/venv/lib/python3.10/site-packages/gradio/blocks.py", line 1296, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "/home/kaminaduck/code/ai-holodeck/venv/lib/python3.10/site-packages/anyio/to_thread.py", line 56, in run_sync
    return await get_async_backend().run_sync_in_worker_thread(
  File "/home/kaminaduc